In [1]:
# ============================================================
# ✅ MODEL PICKLE CONFIGURATION (CHANGE THIS CELL ONLY)
# ============================================================
MODEL_PKL = "/kaggle/input/network-snapshot-0002097-0-100-karrasrho2/pytorch/default/1/network-snapshot-0002097-0.100_karrasrho_noiseinject.pkl"

# Example alternatives you’ll switch between in 5 notebooks:
# MODEL_PKL = "/kaggle/input/network-snapshot-last-linear-unconditional/..."
# MODEL_PKL = "/kaggle/input/network-snapshot-last-cosine-unconditional/..."
# MODEL_PKL = "/kaggle/input/network-snapshot-last-log-unconditional/..."
# MODEL_PKL = "/kaggle/input/network-snapshot-last-quadratic-unconditional/..."


In [2]:
# ============================================================
# ✅ SETUP
# ============================================================
!git clone https://github.com/NVlabs/edm2.git /kaggle/working/edm2
%cd /kaggle/working/edm2
!pip install click tqdm psutil scipy pillow --quiet

Cloning into '/kaggle/working/edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 24.59 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/kaggle/working/edm2


In [3]:
# ============================================================
# 📦 Step 1: Count test images dynamically
# ============================================================
import os
test_dir = "/kaggle/input/celeva-64x64-dataset/celeba64/test"
num_test_imgs = sum(1 for f in os.listdir(test_dir) if f.lower().endswith((".png", ".jpg", ".jpeg")))
print(f"✅ Found {num_test_imgs} test images.")
with open("/kaggle/working/num_test.txt", "w") as f:
    f.write(str(num_test_imgs))

✅ Found 19962 test images.


In [4]:
# ============================================================
# 🖼️ Step 2: Generate images using selected model
# ============================================================
num = int(open("/kaggle/working/num_test.txt").read().strip())
print(f"✅ Generating {num} images with model: {MODEL_PKL}")

os.system(
    f"python /kaggle/working/edm2/generate_images.py "
    f"--net={MODEL_PKL} "
    f"--outdir=/kaggle/working/generated-test-fid "
    f"--seeds=0-{num-1} "
    f"--batch=32 --steps=32 "
    f"--sigma_min=0.002 --sigma_max=80 --rho=7"
)
print("✅ Generation completed.")

✅ Generating 19962 images with model: /kaggle/input/network-snapshot-0002097-0-100-karrasrho2/pytorch/default/1/network-snapshot-0002097-0.100_karrasrho_noiseinject.pkl


[rank0]:[W1114 15:42:53.159473616 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


Loading main network from /kaggle/input/network-snapshot-0002097-0-100-karrasrho2/pytorch/default/1/network-snapshot-0002097-0.100_karrasrho_noiseinject.pkl ...
Setting up StandardRGBEncoder...
Generating 19962 images...


  0%|          | 0/624 [00:00<?, ?batch/s]/kaggle/working/edm2/generate_images.py:235: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  PIL.Image.fromarray(image, 'RGB').save(os.path.join(image_dir, f'{seed:06d}.png'))
100%|██████████| 624/624 [3:22:39<00:00, 19.49s/batch]


✅ Generation completed.


In [5]:
# ============================================================
# 📊 Step 3: Compute Reference Stats (for FID + FD_DINOv2)
# ============================================================
os.system(
    "python /kaggle/working/edm2/calculate_metrics.py ref "
    "--data=/kaggle/input/celeva-64x64-dataset/celeba64/test "
    "--dest=/kaggle/working/celeba64-ref-test.pkl "
    "--metrics=fid,fd_dinov2 "
    "--batch=64"
)

[rank0]:[W1114 19:06:06.485320128 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/dnnlib/torch_hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /root/.cache/dnnlib/torch_hub/checkpoints/dinov2_vitl14_pretrain.pth
  1%|▏         | 16.9M/1.13G [00:00<00:06, 172MB/s]

Loading images from /kaggle/input/celeva-64x64-dataset/celeba64/test ...
Setting up InceptionV3Detector...
Setting up DINOv2Detector...


100%|██████████| 1.13G/1.13G [00:03<00:00, 326MB/s]


Calculating feature statistics...


100%|██████████| 312/312 [20:24<00:00,  3.92s/batch]


0

In [6]:
# ============================================================
# ⚙️ Step 4: Patch FID script to match sample count
# ============================================================
num = int(open("/kaggle/working/num_test.txt").read().strip())
os.system(f'sed -i "s/default=50000/default={num}/" /kaggle/working/edm2/calculate_metrics.py')
print(f"✅ Patched FID script to expect {num} images.")

✅ Patched FID script to expect 19962 images.


In [7]:
# ============================================================
# 📈 Step 5: Compute both FID and FD_DINOv2
# ============================================================
num = int(open("/kaggle/working/num_test.txt").read().strip())
cmd = (
    f"python /kaggle/working/edm2/calculate_metrics.py calc "
    f"--images=/kaggle/working/generated-test-fid "
    f"--ref=/kaggle/working/celeba64-ref-test.pkl "
    f"--metrics=fid,fd_dinov2 "
    f"--num={num} "
    f"--batch=64"
)
os.system(cmd)

[rank0]:[W1114 19:26:47.441022187 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


Loading feature statistics from /kaggle/working/celeba64-ref-test.pkl ...
Loading images from /kaggle/working/generated-test-fid ...
Setting up InceptionV3Detector...
Setting up DINOv2Detector...
Calculating feature statistics...


100%|██████████| 312/312 [20:24<00:00,  3.92s/batch]


Calculating fid...
fid = 10.8144
Calculating fd_dinov2...
fd_dinov2 = 452.299


0

In [8]:
# ============================================================
# 🧾 Step 6: Parse and save results
# ============================================================
import re, subprocess

result = subprocess.getoutput(cmd)
fid_match = re.search(r"fid\s*=\s*([\d.]+)", result)
fd_dino_match = re.search(r"fd_dinov2\s*=\s*([\d.]+)", result)

with open("/kaggle/working/fid_fd_results.txt", "w") as f:
    if fid_match:
        f.write(f"FID = {fid_match.group(1)}\n")
    if fd_dino_match:
        f.write(f"FD_DINOv2 = {fd_dino_match.group(1)}\n")

print(result)
print("✅ Metrics saved to fid_fd_results.txt")

[rank0]:[W1114 19:47:27.348529411 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Loading feature statistics from /kaggle/working/celeba64-ref-test.pkl ...
Loading images from /kaggle/working/generated-test-fid ...
Setting up InceptionV3Detector...
Setting up DINOv2Detector...
Calculating feature statistics...

100%|██████████| 312/312 [20:26<00:00,  3.93s/batch]
Calculating fid...
fid = 10.8144
Calculating fd_dinov2...
fd_dinov2 = 452.299
✅ Metrics saved to fid_fd_results.txt
